In [ ]:
"""
This file is the entry point for the project
"""
# Begin by importing external libraries using path_setup
# NOTE : Must be ran first, thus PEP8-E402
import path_setup
path_setup.path_setup()
import sys  # noqa E402
import matplotlib.pyplot as plt # noqa E402
import numpy as np # noqa E402
import datetime as dt # noqa E402
from LatticeClass_F import lattice_class as lt # noqa E402
import PrintException as PE  # noqa E402
import input_funcs as inF  # noqa E402
from random import random  # noqa E402
# Shebang line for interactive output in vs_code, comment this out if you have troubles running the notebook\n",
# %matplotlib widget

def rand_time() -> int:
    out = int(dt.datetime.now().strftime('%s'))
    sys.stdout.write(f"Time Seed = {out}\n")
    return(int(dt.datetime.now().strftime('%s')))

In [ ]:
N = 15
M = 15
BJs = np.arange(0.1, 2, 0.05)
total_time = 1000
BJ = 0.1

size = [N, M]
lt_a = lt(1, size)
lt_b = lt(1, size, [[1, 0], [0.5, np.sqrt(3)/2]])
lt_c = lt(1, size, [[0.5, np.sqrt(3)/2], [0.5, -np.sqrt(3)/2]])
lt_d = lt(1, size, [[0.128, np.e], [3.02398, -np.e]])

# output = input('Enter 0 for seeded random or 1 for time based:')
output = '0'
if output == '0':
    print("option 0 chosen..\n")
    # DOCtest seed = 1644121893
    seed = 1644121893
    lt_a.randomize(voids=True , probs=[23, 50], rand_seed=seed, quiet=False)
    lt_b.randomize(voids=False, probs=[23, 50], rand_seed=seed)
    lt_c.randomize(voids=False, probs=[23, 50], rand_seed=seed)
    lt_d.randomize(voids=False, probs=[23, 50], rand_seed=seed)

else:
    print("option 1 chosen.\n")
    lt_a.randomize(voids=True , probs=[random(100), random(100)], rand_seed=rand_time(), quiet=False)
    lt_b.randomize(voids=False, probs=[random(100), random(100)], rand_seed=rand_time(), quiet=False)
    lt_c.randomize(voids=False, probs=[random(100), random(100)], rand_seed=rand_time(), quiet=False)
    lt_d.randomize(voids=False, probs=[random(100), random(100)], rand_seed=rand_time(), quiet=False)


In [ ]:
lt_a.display()
lt_b.display()
lt_c.display()
lt_d.display()

# Metropolis Algorithm test
Uncomment if you want to test these. The metropolis algorithm gets ran anyways in the next section.

In [ ]:
# total_time = 1000
# BJ = 1
# SE_mtx = lt_a.metropolis(total_time, BJ, quiet=False)
# plot_metrop(SE_mtx, BJ, size)
# SE_mtx = lt_b.metropolis(total_time, BJ)
# plot_metrop(SE_mtx, BJ, size)
# SE_mtx = lt_c.metropolis(total_time, BJ)
# plot_metrop(SE_mtx, BJ, size)
# SE_mtx = lt_d.metropolis(total_time, BJ)
# plot_metrop(SE_mtx, BJ, size)

In [ ]:
a, b, c = lt_a.get_spin_energy(BJs, total_time)

In [ ]:
a, b, c = lt_b.get_spin_energy(BJs, total_time)


In [ ]:
a, b, c = lt_c.get_spin_energy(BJs, total_time)

In [ ]:
a, b, c = lt_d.get_spin_energy(BJs, total_time)